# Using the LQR Algorithm to Control a Robot

| Package       | Version |
|---------------|---------|
| numpy         | < 2.0   |
| scipy         |         |
| matplotlib    |         |
| grpcio        |         |
| grpcio-tools  |         |
| torch         |         |
| Level Name    | humanoid_test_03 |

This example is ported from the LRQ example that comes with the Mujoco project. In this example, you can learn how to port from Mujoco to OrcaGym.

Install the above packages, ensuring compatibility between numpy and torch by installing numpy < version 2.0.

To use the LQR (Linear Quadratic Regulator) algorithm to control the robot and maintain its stable standing, you need to follow these steps:

1. **Establish the Robot Model and LQR Controller**: First, you need to establish the state-space model of the robot and design the LQR controller based on this model.
2. **Collect the Current State of the Robot**: Use the gRPC interface to get the current qpos and qvel of the robot.
3. **Calculate the Control Input**: Use the LQR algorithm to compute the control input.
4. **Send the Control Input**: Send the calculated control input to the robot through the gRPC interface.


In [ ]:
import os
import sys

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

# Add the project root directory to PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)

import grpc
import importlib
import orca_gym.orca_gym as orca_gym
from orca_gym.orca_gym import OrcaGym


In [ ]:
import asyncio
import nest_asyncio

import numpy as np
import scipy.linalg
from datetime import datetime


# Nested Event Loops
nest_asyncio.apply()

config = {
    "fps-30": {
        "frame_time": 0.03333333333333333,
    },
    "fps-60": {
        "frame_time": 0.016666666666666666,
    },
    "fps-120": {
        "frame_time": 0.008333333333333333,
    },
    "fps-2400": {
        "frame_time": 0.0004166666666666667,
    }
}


ctrl0 = None
qpos0 = None

## Finding Optimal Control Initialization Parameters

1. **Obtain Model Information and Weight**:

* Query model information and gravity configuration via the asynchronous gRPC interface to calculate the robot's weight.

2. **Set High-Precision Simulation Step**:

* Set the simulation step to a high precision value to accurately find the reaction force at balance.

3. **Initialize Scene and Controller**:

* Initialize the scene and set the controller to zero.

3. **Record Data**:

* Continuously query and record position, velocity, reaction force, and actuator matrix data at each time step.

4. **Find the Balance Frame**:

* Identify the time step where the vertical reaction force is closest to the robot's weight by comparing the recorded reaction force values, and determine the balance frame.

5. **Calculate Control Torque**:

* Compute the control torque using the reaction force at the balance frame and the pseudo-inverse of the actuator matrix, ensuring it is negative to reflect the reaction force.

6. **Return Results**:

* Return the position, velocity, reaction force, and actuator matrix of the balance frame.


In [ ]:
import matplotlib.pyplot as plt

async def find_balance_frame(sim):

    # Get model information
    model_info = await sim.query_model_info()
    print("Model info: ", model_info)
    nv = model_info['nv']
    

    # Calculate the weight of the humanoid and get the optimal force value
    mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
    opt_config = await sim.query_opt_config()

    gravite = opt_config["gravity"]
    robot_weight = mass * np.linalg.norm(gravite)
    print("Weight: ", robot_weight)

    # Set the simulation timestep to 2000 fps to accurately find the reaction force at balance
    await sim.set_opt_timestep(config["fps-2400"]["frame_time"])

    await sim.pause_simulation()

    # Initialize the scene
    await sim.load_initial_frame()

    await sim.load_keyframe("stand_on_left_leg")

    qpos, qvel = await sim.query_all_qpos_and_qvel()
    qpos[2] += 0.01  # Offset the height by 0.01.
    await sim.set_qpos(qpos)
    
    # Set the controller to 0
    await sim.set_ctrl(np.zeros(model_info['nu']))

    await sim.resume_simulation()

    qfrc_inverse_list = []
    qpos_list = []
    qvel_list = []
    actuator_moment_list = []
    frame_count = 7000
    for frame in range(frame_count):            
        # Offset the height by `offset`.
        qpos, qvel = await sim.query_all_qpos_and_qvel()

        actuator_moment = await sim.query_actuator_moment()

        await sim.mj_inverse()
        qfrc_inverse = await sim.query_qfrc_inverse()

        qfrc_inverse_list.append(qfrc_inverse)
        qpos_list.append(qpos)
        qvel_list.append(qvel)
        actuator_moment_list.append(actuator_moment)

    # Find the height-offset at which the vertical force is smallest.
    qfrc_inverse_list = np.array(qfrc_inverse_list)
    vertical_forces = np.abs(qfrc_inverse_list[:, 2])
    balance_frame = np.argmin(np.abs(vertical_forces - robot_weight))

    # Plot the relationship.
    plt.figure(figsize=(10, 6))
    plt.plot(np.arange(frame_count), vertical_forces, linewidth=3)
    plt.axvline(x=balance_frame, color='red', linestyle='--')
    plt.axhline(y=robot_weight, color='green', linestyle='--')
    plt.xlabel('frames ')
    plt.ylabel('Vertical force (N)')
    plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
    plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    plt.minorticks_on()
    plt.title(f'Found best vertical force at frame {balance_frame}.')
    plt.show()        

    return qpos_list[balance_frame], qvel_list[balance_frame], qfrc_inverse_list[balance_frame], actuator_moment_list[balance_frame]


async def calc_control_0():
    ROBOT_NAMES = ["Robot_01"]
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub, ROBOT_NAMES)
        
        await sim.query_agents()

        global ctrl0, qpos0

        # Get model information
        model_info = await sim.query_model_info()

        # Run the simulation once to get the array of reaction forces at balance
        qpos0, qvel0, qfrc0, actuator_moment = await find_balance_frame(sim)
        print(f"Balance Frame: qpos0 = {qpos0}, qvel0 = {qvel0}, qfrc0 = {qfrc0}, actuator_moment = {actuator_moment}")
        print(f"Length of qfrc0: {len(qfrc0)}")

        ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
        ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
        
        # Convert to negative values because the reaction forces are the forces exerted by the robot on the environment
        ctrl0 = np.multiply(ctrl0, -1) 

        # Set default qpos
        qpos0 = np.array(qpos0)


# if __name__ == "__main__":
#     asyncio.run(calc_control_0())


## Finding the control setpoint using inverse dynamics

MuJoCo's forward dynamics function `mj_forward`, which we used above in order to propagate derived quantities, computes the acceleration given the state and all the forces in the system, some of which are created by the actuators.

The inverse dynamics function takes the acceleration as *input*, and computes the forces required to create the acceleration. Uniquely, MuJoCo's [fast inverse dynamics](https://doi.org/10.1109/ICRA.2014.6907751) takes into account all constraints, including contacts. Let's see how it works.

We'll call the forward dynamics at our desired position setpoint, set the acceleration in `data.qacc` to 0, and call the inverse dynamics:

In [ ]:
async def calc_control_0_by_inverse(floor_offset = 0):    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)
    
        model_info = await sim.query_model_info()
        opt_config = await sim.query_opt_config()

        global ctrl0, qpos0

        await sim.pause_simulation()
        await sim.load_initial_frame()

        # mujoco.mj_resetDataKeyframe(model, data, 1)
        # mujoco.mj_forward(model, data)
        # data.qacc = 0  # Assert that there is no the acceleration.
        # mujoco.mj_inverse(model, data)
        # print(data.qfrc_inverse)

        
        await sim.load_keyframe("stand_on_left_leg")
        await sim.mj_forward()
        qacc = np.zeros(model_info['nv'])
        await sim.set_qacc(qacc)
        await sim.mj_inverse()
        qfrc_inverse = await sim.query_qfrc_inverse()
        print("Qfrc_inverse: ", qfrc_inverse)
        

        # height_offsets = np.linspace(-0.001, 0.001, 2001)
        # vertical_forces = []
        # for offset in height_offsets:
        #     mujoco.mj_resetDataKeyframe(model, data, 1)
        #     mujoco.mj_forward(model, data)
        #     data.qacc = 0
        #     # Offset the height by `offset`.
        #     data.qpos[2] += offset
        #     mujoco.mj_inverse(model, data)
        #     vertical_forces.append(data.qfrc_inverse[2])


        height_offsets = np.linspace(-0.001, 0.001, 2001)
        vertical_forces = []
        for offset in height_offsets:
            await sim.load_keyframe("stand_on_left_leg")
            await sim.mj_forward()
            qacc = np.zeros(model_info['nv'])
            await sim.set_qacc(qacc)

            qpos, _, _ = await sim.query_all_qpos_qvel_qacc()
            qpos[2] += offset + floor_offset  # Offset the height by `offset`.
            await sim.set_qpos(qpos)
            await sim.mj_inverse()
            qfrc_inverse = await sim.query_qfrc_inverse()
            vertical_forces.append(qfrc_inverse[2])

        # Find the height-offset at which the vertical force is smallest.
        idx = np.argmin(np.abs(vertical_forces))
        best_offset = height_offsets[idx] + floor_offset
        print(f'Smallest vertical force found {vertical_forces[idx]:.4f}n.')

        # Plot the relationship.
        plt.figure(figsize=(10, 6))
        plt.plot(height_offsets * 1000, vertical_forces, linewidth=3)
        # Red vertical line at offset corresponding to smallest vertical force.
        plt.axvline(x=best_offset*1000, color='red', linestyle='--')
        # Green horizontal line at the humanoid's weight.
        # weight = model.body_subtreemass[1]*np.linalg.norm(model.opt.gravity)
        weight = await sim.query_body_subtreemass_by_name("Robot_01_torso") * np.linalg.norm(opt_config["gravity"])

        plt.axhline(y=weight, color='green', linestyle='--')
        plt.xlabel('Height offset (mm)')
        plt.ylabel('Vertical force (N)')
        plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
        plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
        plt.minorticks_on()
        plt.title(f'Smallest vertical force '
                f'found at offset {best_offset*1000:.4f}mm.')
        plt.show()

        # mujoco.mj_resetDataKeyframe(model, data, 1)
        # mujoco.mj_forward(model, data)
        # data.qacc = 0
        # data.qpos[2] += best_offset
        # qpos0 = data.qpos.copy()  # Save the position setpoint.
        # mujoco.mj_inverse(model, data)
        # qfrc0 = data.qfrc_inverse.copy()
        # print('desired forces:', qfrc0)

        await sim.load_keyframe("stand_on_left_leg")
        await sim.mj_forward()
        qacc = np.zeros(model_info['nv'])
        await sim.set_qacc(qacc)
        qpos, _, _ = await sim.query_all_qpos_qvel_qacc()
        qpos[2] += best_offset
        qpos0 = qpos
        await sim.set_qpos(qpos)
        await sim.mj_inverse()
        qfrc0 = await sim.query_qfrc_inverse()
        print('desired forces:', qfrc0)
    
        # ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(data.actuator_moment)
        # ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
        # print('control setpoint:', ctrl0)

        actuator_moment = await sim.query_actuator_moment()
        ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
        ctrl0 = ctrl0.flatten()
        print('control setpoint:', ctrl0)


        # data.ctrl = ctrl0
        # mujoco.mj_forward(model, data)
        # print('actuator forces:', data.qfrc_actuator)

        await sim.set_ctrl(ctrl0)
        await sim.mj_forward()
        qfrc_actuator = await sim.query_qfrc_actuator()
        print('actuator forces:', qfrc_actuator)

        await sim.resume_simulation()

if __name__ == "__main__":
    asyncio.run(calc_control_0_by_inverse(-0.0130))

## Test the Control Program Based on ctrl0

* Check if the humanoid has gained strength.

In [ ]:
async def test_ctrl0():
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)

        
        global ctrl0
        global qpos0
        # The ctrl0 has been calculated above.
        print("Control 0: ", ctrl0)
        print("qpos0: ", qpos0)

        mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
        opt_config = await sim.query_opt_config()        
        print("opt config : ", opt_config)

        # Set the simulation rate to 60fps
        await sim.set_opt_timestep(config["fps-60"]["frame_time"])

        # Get the model information
        model_info = await sim.query_model_info()
        print("Model info: ", model_info)

        nu = model_info['nu']  # Alias for the number of actuators.
        nv = model_info['nv'] # Shortcut for the number of DoFs.

        # Initialize the scene
        await sim.pause_simulation()

        await sim.load_initial_frame()

        await sim.load_keyframe("stand_on_left_leg")

        # await sim.set_qpos(stub, qpos0)
        await sim.set_ctrl(ctrl0)

        await sim.resume_simulation()


if __name__ == "__main__":
    asyncio.run(test_ctrl0())

## Implement the LQR Control Algorithm

1. **Keyframe Transformation**:
    * Implement keyframe transformation based on the actual joint order and the keyframes defined in the original XML, adapting the keyframes to the current scene to achieve single-leg standing.

2. **Port Mujoco Example Code**:
    * Port the example code from the Mujoco project to implement LQR control.

In [ ]:
async def main():
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)

        mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
        opt_config = await sim.query_opt_config()        
        print("opt config : ", opt_config)

        frame_time = config["fps-120"]["frame_time"]

        # Set simulation rate to 60fps
        await sim.set_opt_timestep(frame_time)

        # Get model information
        model_info = await sim.query_model_info()
        print("Model info: ", model_info)

        # Get all joint names
        joint_names = await sim.query_joint_names()
        print("Joint names: ", joint_names)

        # Get the starting indices of degrees of freedom for all joints
        joint_dofadrs = await sim.query_joint_dofadr(joint_names)
        print("Joint dofadrs: ", joint_dofadrs)

        # Get the control objects
        actuator_dict = await sim.query_all_actuators()
        print(f"Actuator_info: {actuator_dict}")

        gear_ratios = {key: value['GearRatio'] for key, value in actuator_dict.items()}
        print(f"Gear ratios: {gear_ratios}")

        nu = model_info['nu']  # Alias for the number of actuators.
        R = np.eye(nu)

        nv = model_info['nv'] # Shortcut for the number of DoFs.

        # Initialize the scene
        await sim.pause_simulation()
        global ctrl0
        await sim.set_ctrl(ctrl0)

        # Execute mj_forward during initialization
        await sim.load_initial_frame()

        await sim.load_keyframe("stand_on_left_leg")
        
        # # Load the keyframe with name "stand_on_left_leg"
        # result = await load_keyframe_to_server(stub, get_qpos_for_keyframe("squat", joint_names, prefix="Robot_01_"))
        # if not result:
        #     print("Failed to load keyframe stand_on_left_leg")
        #     return

        # Initialize the robot with the found balance position
        await sim.set_qpos(qpos0)

        # Get the Jacobian for the root body (torso) CoM.        
        await sim.mj_forward()
        jac_com = await sim.mj_jac_subtree_com("Robot_01_torso")
        print("Jacobian CoM: ", jac_com)

        # Get the Jacobian for the left foot.
        jac_foot, _ = await sim.mj_jac_body_com("Robot_01_foot_left")
        print("Jacobian foot: ", jac_foot)

        jac_diff = jac_com - jac_foot
        Qbalance = jac_diff.T @ jac_diff
        print("Qbalance: ", Qbalance)

        # Generate a dictionary based on joint names and starting indices of degrees of freedom
        joint_dofadr_map = {name: dofadr for name, dofadr in zip(joint_names, joint_dofadrs)}

        # Get the indices of relevant sets of joints
        root_dofs = range(6)
        body_dofs = range(6, nv)
        abdomen_dofs = [
            joint_dofadr_map[name]
            for name in joint_names
            if 'abdomen' in name
            and 'z' not in name
        ]
        left_leg_dofs = [
            joint_dofadr_map[name]
            for name in joint_names
            if 'left' in name
            and any(keyword in name for keyword in ['hip', 'knee', 'ankle'])
            and 'z' not in name
        ]
        balance_dofs = abdomen_dofs + left_leg_dofs
        other_dofs = np.setdiff1d(body_dofs, balance_dofs)

        # Cost coefficients.
        BALANCE_COST        = 1000  # Balancing.
        BALANCE_JOINT_COST  = 3     # Joints required for balancing.
        OTHER_JOINT_COST    = .3    # Other joints.

        # Construct the Qjoint matrix.
        Qjoint = np.eye(nv)
        Qjoint[root_dofs, root_dofs] *= 0  # Don't penalize free joint directly.
        Qjoint[balance_dofs, balance_dofs] *= BALANCE_JOINT_COST
        Qjoint[other_dofs, other_dofs] *= OTHER_JOINT_COST

        # Construct the Q matrix for position DoFs.
        Qpos = BALANCE_COST * Qbalance + Qjoint

        # No explicit penalty for velocities.
        Q = np.block([[Qpos, np.zeros((nv, nv))],
                    [np.zeros((nv, 2*nv))]])

        # Set the initial state and control.
        await sim.load_keyframe("stand_on_left_leg")
        await sim.set_qpos(qpos0)
        await sim.set_ctrl(ctrl0)
        await sim.mj_forward()

        # Allocate the A and B matrices, compute them.
        epsilon = 1e-6
        flg_centered = True
        A, B, _, _ = await sim.mjd_transition_fd(epsilon, flg_centered)

        # Solve discrete Riccati equation.
        P = scipy.linalg.solve_discrete_are(A, B, Q, R)

        # Compute the feedback gain matrix K.
        K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

        # Allocate position difference dq.
        dq = np.zeros(nv)

        # Control loop
        for i in range(1200):
            start_time = datetime.now()

            # Get current joint positions and velocities
            qpos, qvel, _ = await sim.query_all_qpos_qvel_qacc()

            # Get state difference dx.
            dq = await sim.mj_differentiate_pos(qpos0, qpos)
            dx = np.hstack((dq, qvel)).T

            # LQR control law.
            data_ctrl = ctrl0 - K @ dx

            # Set the control signal.
            await sim.set_ctrl(data_ctrl)

            await sim.mj_step(1)

            # Wait for the next control cycle
            end_time = datetime.now()
            delta_time = (end_time - start_time).total_seconds()
            if delta_time < frame_time:
                await asyncio.sleep(frame_time - delta_time)
            else:
                print(f"Control loop took too long: {delta_time} seconds")

        # Resume free execution
        await sim.resume_simulation()

if __name__ == '__main__':
    asyncio.run(main())
